In [106]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from scipy.stats import zscore
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OrdinalEncoder,OneHotEncoder,MinMaxScaler
from sklearn.ensemble import BaggingClassifier
from sklearn.model_selection import cross_val_score

In [2]:
df=pd.read_csv("heart.csv")
df.head(10)

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40,M,ATA,140,289,0,Normal,172,N,0.0,Up,0
1,49,F,NAP,160,180,0,Normal,156,N,1.0,Flat,1
2,37,M,ATA,130,283,0,ST,98,N,0.0,Up,0
3,48,F,ASY,138,214,0,Normal,108,Y,1.5,Flat,1
4,54,M,NAP,150,195,0,Normal,122,N,0.0,Up,0
5,39,M,NAP,120,339,0,Normal,170,N,0.0,Up,0
6,45,F,ATA,130,237,0,Normal,170,N,0.0,Up,0
7,54,M,ATA,110,208,0,Normal,142,N,0.0,Up,0
8,37,M,ASY,140,207,0,Normal,130,Y,1.5,Flat,1
9,48,F,ATA,120,284,0,Normal,120,N,0.0,Up,0


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 918 entries, 0 to 917
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Age             918 non-null    int64  
 1   Sex             918 non-null    object 
 2   ChestPainType   918 non-null    object 
 3   RestingBP       918 non-null    int64  
 4   Cholesterol     918 non-null    int64  
 5   FastingBS       918 non-null    int64  
 6   RestingECG      918 non-null    object 
 7   MaxHR           918 non-null    int64  
 8   ExerciseAngina  918 non-null    object 
 9   Oldpeak         918 non-null    float64
 10  ST_Slope        918 non-null    object 
 11  HeartDisease    918 non-null    int64  
dtypes: float64(1), int64(6), object(5)
memory usage: 86.2+ KB


In [4]:
df.describe()

,Age,RestingBP,Cholesterol,FastingBS,MaxHR,Oldpeak,HeartDisease
count,918.000000,918.000000,918.000000,918.000000,918.000000,918.000000,918.000000
mean,53.510893,132.396514,198.799564,0.233115,136.809368,0.887364,0.553377
std,9.432617,18.514154,109.384145,0.423046,25.460334,1.066570,0.497414
min,28.000000,0.000000,0.000000,0.000000,60.000000,-2.600000,0.000000
25%,47.000000,120.000000,173.250000,0.000000,120.000000,0.000000,0.000000
50%,54.000000,130.000000,223.000000,0.000000,138.000000,0.600000,1.000000
75%,60.000000,140.000000,267.000000,0.000000,156.000000,1.500000,1.000000
max,77.000000,200.000000,603.000000,1.000000,202.000000,6.200000,1.000000


In [5]:
cols=['RestingBP','Cholesterol','MaxHR','Oldpeak']
for i in cols:
    df[f'Z-score_{i}'] = zscore(df[i])


In [6]:
df.head(10)

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease,Z-score_RestingBP,Z-score_Cholesterol,Z-score_MaxHR,Z-score_Oldpeak
0,40,M,ATA,140,289,0,Normal,172,N,0.0,Up,0,0.410909,0.825070,1.382928,-0.832432
1,49,F,NAP,160,180,0,Normal,156,N,1.0,Flat,1,1.491752,-0.171961,0.754157,0.105664
2,37,M,ATA,130,283,0,ST,98,N,0.0,Up,0,-0.129513,0.770188,-1.525138,-0.832432
3,48,F,ASY,138,214,0,Normal,108,Y,1.5,Flat,1,0.302825,0.139040,-1.132156,0.574711
4,54,M,NAP,150,195,0,Normal,122,N,0.0,Up,0,0.951331,-0.034755,-0.581981,-0.832432
5,39,M,NAP,120,339,0,Normal,170,N,0.0,Up,0,-0.669935,1.282424,1.304332,-0.832432
6,45,F,ATA,130,237,0,Normal,170,N,0.0,Up,0,-0.129513,0.349422,1.304332,-0.832432
7,54,M,ATA,110,208,0,Normal,142,N,0.0,Up,0,-1.210356,0.084157,0.203982,-0.832432
8,37,M,ASY,140,207,0,Normal,130,Y,1.5,Flat,1,0.410909,0.075010,-0.267596,0.574711
9,48,F,ATA,120,284,0,Normal,120,N,0.0,Up,0,-0.669935,0.779335,-0.660578,-0.832432


In [7]:
df.shape

(918, 16)

In [8]:
indices=df[(df['Z-score_RestingBP'].abs() > 3) | (df['Z-score_Cholesterol'].abs() > 3) | (df['Z-score_Oldpeak'].abs() > 3)].index

In [9]:
df.drop(indices,axis='index',inplace=True)
df.shape

(900, 16)

In [10]:
df.drop(['Z-score_RestingBP','Z-score_Cholesterol','Z-score_MaxHR','Z-score_Oldpeak'],axis='columns',inplace =True)
df.head(10)

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40,M,ATA,140,289,0,Normal,172,N,0.0,Up,0
1,49,F,NAP,160,180,0,Normal,156,N,1.0,Flat,1
2,37,M,ATA,130,283,0,ST,98,N,0.0,Up,0
3,48,F,ASY,138,214,0,Normal,108,Y,1.5,Flat,1
4,54,M,NAP,150,195,0,Normal,122,N,0.0,Up,0
5,39,M,NAP,120,339,0,Normal,170,N,0.0,Up,0
6,45,F,ATA,130,237,0,Normal,170,N,0.0,Up,0
7,54,M,ATA,110,208,0,Normal,142,N,0.0,Up,0
8,37,M,ASY,140,207,0,Normal,130,Y,1.5,Flat,1
9,48,F,ATA,120,284,0,Normal,120,N,0.0,Up,0


In [11]:
df['ST_Slope'].unique()

array(['Up', 'Flat', 'Down'], dtype=object)

In [12]:
x=df.drop(["HeartDisease"],axis='columns')
y=df['HeartDisease']

In [13]:
ct=ColumnTransformer(transformers=[
    ('tnf1',OrdinalEncoder(categories=[['Up','Flat','Down']]),['ST_Slope']),
    ('tnf2',OneHotEncoder(sparse_output=False,drop='first'),['Sex','ChestPainType','ExerciseAngina','RestingECG'])
],remainder='passthrough'
)
x=ct.fit_transform(x)
x[:10,:]

array([[  0. ,   1. ,   1. ,   0. ,   0. ,   0. ,   1. ,   0. ,  40. ,
        140. , 289. ,   0. , 172. ,   0. ],
       [  1. ,   0. ,   0. ,   1. ,   0. ,   0. ,   1. ,   0. ,  49. ,
        160. , 180. ,   0. , 156. ,   1. ],
       [  0. ,   1. ,   1. ,   0. ,   0. ,   0. ,   0. ,   1. ,  37. ,
        130. , 283. ,   0. ,  98. ,   0. ],
       [  1. ,   0. ,   0. ,   0. ,   0. ,   1. ,   1. ,   0. ,  48. ,
        138. , 214. ,   0. , 108. ,   1.5],
       [  0. ,   1. ,   0. ,   1. ,   0. ,   0. ,   1. ,   0. ,  54. ,
        150. , 195. ,   0. , 122. ,   0. ],
       [  0. ,   1. ,   0. ,   1. ,   0. ,   0. ,   1. ,   0. ,  39. ,
        120. , 339. ,   0. , 170. ,   0. ],
       [  0. ,   0. ,   1. ,   0. ,   0. ,   0. ,   1. ,   0. ,  45. ,
        130. , 237. ,   0. , 170. ,   0. ],
       [  0. ,   1. ,   1. ,   0. ,   0. ,   0. ,   1. ,   0. ,  54. ,
        110. , 208. ,   0. , 142. ,   0. ],
       [  1. ,   1. ,   0. ,   0. ,   0. ,   1. ,   1. ,   0. ,  37. ,
        1

In [14]:
x.shape

(900, 14)

In [15]:
scaler=MinMaxScaler()
x_scale=scaler.fit_transform(x)
x_scale[:10,:]

array([[0.        , 1.        , 1.        , 0.        , 0.        ,
        0.        , 1.        , 0.        , 0.24489796, 0.57142857,
        0.55791506, 0.        , 0.78873239, 0.33333333],
       [0.5       , 0.        , 0.        , 1.        , 0.        ,
        0.        , 1.        , 0.        , 0.42857143, 0.76190476,
        0.34749035, 0.        , 0.67605634, 0.5       ],
       [0.        , 1.        , 1.        , 0.        , 0.        ,
        0.        , 0.        , 1.        , 0.18367347, 0.47619048,
        0.54633205, 0.        , 0.26760563, 0.33333333],
       [0.5       , 0.        , 0.        , 0.        , 0.        ,
        1.        , 1.        , 0.        , 0.40816327, 0.55238095,
        0.41312741, 0.        , 0.33802817, 0.58333333],
       [0.        , 1.        , 0.        , 1.        , 0.        ,
        0.        , 1.        , 0.        , 0.53061224, 0.66666667,
        0.37644788, 0.        , 0.43661972, 0.33333333],
       [0.        , 1.        , 0. 

In [18]:
x_train,x_test,y_train,y_test=train_test_split(x_scale,y,test_size=0.2)

## Standalone SVM 

In [114]:
model=SVC(C=1,kernel='rbf')
model.fit(x_train,y_train)
model.score(x_test,y_test)
scores=cross_val_score(model,x_scale,y,cv=5)
scores.mean()

np.float64(0.8177777777777777)

## Bagged SVM

In [103]:
bag_svm=BaggingClassifier(estimator=SVC(C=1),n_estimators=100,max_samples=0.6,oob_score=True,random_state=0)
bag_svm.fit(x_train,y_train)

BaggingClassifier(estimator=SVC(C=1), max_samples=0.6, n_estimators=100,
                  oob_score=True, random_state=0)

In [104]:
bag_svm.oob_score_

0.8583333333333333

In [105]:
bag_svm.score(x_test,y_test)

0.8277777777777777

## Optimising model Parameters

In [64]:
opt_model=GridSearchCV(estimator=bag_svm,param_grid={
    'estimator__C':[0.1,0.2,0.3,0.4,0.5],
    'estimator__gamma':['scale','auto']
})

In [65]:
opt_model.fit(x_train,y_train)

GridSearchCV(estimator=BaggingClassifier(estimator=SVC(C=0.5), max_samples=0.6,
                                         n_estimators=100, oob_score=True,
                                         random_state=0),
             param_grid={'estimator__C': [0.1, 0.2, 0.3, 0.4, 0.5],
                         'estimator__gamma': ['scale', 'auto']})

In [66]:
opt_model.best_score_

np.float64(0.8541666666666667)

In [67]:
opt_model.best_params_

{'estimator__C': 0.5, 'estimator__gamma': 'scale'}

In [68]:
opt_model.score(x_test,y_test)

0.8166666666666667

In [57]:
scores=cross_val_score(bag_svm,x_scale,y,cv=5)
scores

array([0.86111111, 0.85555556, 0.82777778, 0.81666667, 0.76666667])

In [58]:
scores.mean()

np.float64(0.8255555555555555)

## Bagged Tree

In [109]:
bag_tree=BaggingClassifier(estimator=DecisionTreeClassifier(),n_estimators=100,max_samples=0.6,oob_score=True,random_state=0)
bag_tree.fit(x_train,y_train)

BaggingClassifier(estimator=DecisionTreeClassifier(), max_samples=0.6,
                  n_estimators=100, oob_score=True, random_state=0)

In [111]:
bag_tree.oob_score_

0.8597222222222223

In [110]:
bag_tree.score(x_test,y_test)

0.8333333333333334

In [112]:
scores=cross_val_score(bag_tree,x_scale,y,cv=5)
scores

array([0.86111111, 0.82777778, 0.81666667, 0.82222222, 0.73888889])

In [113]:
scores.mean()

np.float64(0.8133333333333332)

## Random Forest
### Uses Bagging of Decision Trees. But also bags features along with indices

In [116]:
rf_model=RandomForestClassifier()
scores=cross_val_score(rf_model,x_scale,y,cv=5)
scores.mean()

np.float64(0.818888888888889)